Step 1 of Fast ice detection workflow: Normalised Product

ARE Settings:

Queue: Normal
Compute size: Medium
Jobfs size: 10GB

Requires process_image_pairs.py

Conda environment: sam_env

In [1]:
# initial setup 
#Step1: Normprod Step
import os
import gc
import re
import yaml
import shutil
from shapely import Polygon, box
import numpy as np
import pyproj
import rasterio
import rasterio.merge
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
#import Tony_FastIce24
import json
import subprocess
from osgeo import gdal,ogr,osr,gdal_array, gdalconst
gdal.UseExceptions()
import scipy.ndimage
from scipy.ndimage import uniform_filter#, convolve   # ALEX ADDED THIS to get the local SD within the kernel
from scipy.ndimage import gaussian_filter, generic_filter  # And chat added this because it's faster than the two above
import pdb #for debugging.
import datetime
from pathlib import Path
import psutil
import sys
import ast



In [2]:
#Normprod 1.1
## THIS IS FOR CAITLIN EWs - and doing the FULL STACK
### calculate maximal bounds for whole stack for a common projection geot. 

process = psutil.Process(os.getpid())
memory_usage = process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB
print(f"Start: Memory usage: {memory_usage:.2f} MB")

useWholeStack = True   #   else use pair.
findOuterBounds = True   # ie. union,  else innerBounds, ie. intersection.

localDataDir = os.path.join('/','g','data','jk72','gb4219','honours_data','Normprod_Demo') ########## EW DIRECTORY HERE ############

searchstring = "_3031.tif"  ########### Read files with this suffix (sometimes Caitlin changes them) ########

full_filenames = sorted([f for f in Path(localDataDir).iterdir() if f.is_file() and searchstring in f.name])
base_filenames = [f.name for f in full_filenames]

startXs = []
endXs = []
deltaXs = []
startYs = []
endYs = []
deltaYs = []
if useWholeStack:
    useRange = range(len(base_filenames))
    print("Finding whole stack intersection")
else:
    useRange = imPair
    print("Finding pair intersection")
for which in useRange: # Want to find common bounds for whole stack.
    #pdb.set_trace()
    geoGRDi = os.path.join(localDataDir, base_filenames[which])
    print(geoGRDi)
    if os.path.exists(geoGRDi):
        #check extents overlap and crop to intersection with gdal.Warp
        DSi = gdal.Open(geoGRDi, gdal.GA_ReadOnly)
        NXi, NYi, NBi = DSi.RasterXSize, DSi.RasterYSize, DSi.RasterCount
        geoti = DSi.GetGeoTransform()
        startXs.append(geoti[0])
        endXs.append(geoti[0]+geoti[1]*NXi)
        startYs.append(geoti[3])
        endYs.append(geoti[3]+geoti[5]*NYi)  # NB. geoti[5] is -ve.
        deltaXs.append(geoti[1])
        deltaYs.append(geoti[5])
        geoti = None
        DSi = None
# should find actual data extent, not just image bounds. Need to get inside arrays looking for non-NaN extents.
#  For now, just let the compression reduce the file size and leave the actual extent.
#  [X0 P.dx L.dy Y0 P.dx L.dy]

if findOuterBounds:
    newX0 = min(startXs)
    newPdx = sum(deltaXs)/len(deltaXs)
    newY0 = max(startYs)
    newLdy = sum(deltaYs)/len(deltaYs)
    endX = max(endXs)
    endY = min(endYs)  # NB: Y-axis goes negative direction
    stackBounds = (newX0, newY0, endX, endY)
    print("Determined maximal (union) stack bounds.", [round(e, 3) for e in stackBounds])
else:
    #pdb.set_trace()
    newX0 = max(startXs)
    newPdx = sum(deltaXs)/len(deltaXs)
    newY0 = min(startYs)
    newLdy = sum(deltaYs)/len(deltaYs)
    endX = min(endXs)
    endY = max(endYs)  # NB: Y-axis goes negative direction
    stackBounds = (newX0, newY0, endX, endY)
    print("Determined minimal (intersection) stack bounds.", [round(e, 3) for e in stackBounds])
newgeot = [newX0, newPdx, 0, newY0, 0, newLdy]

process = psutil.Process(os.getpid())
memory_usage = process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB
print(f"End: Memory usage: {memory_usage:.2f} MB")


Start: Memory usage: 572.05 MB
Finding whole stack intersection
/g/data/jk72/gb4219/honours_data/Normprod_Demo/S1A__EW___A_20210716T152833_HH_grd_mli_gamma0-rtc_geo_db_3031.tif
/g/data/jk72/gb4219/honours_data/Normprod_Demo/S1A__EW___A_20210728T152834_HH_grd_mli_gamma0-rtc_geo_db_3031.tif
Determined maximal (union) stack bounds. [1815744.144, 1155865.665, 2606917.871, 374040.5]
End: Memory usage: 572.30 MB


In [3]:
# Normprod 1.2. Overlap-trimming
# this is where we can choose a temporal baseline 
# and trim image pairs to their overlapping area if baseline is less or more than a repeat pass
import os
import numpy as np
import gc
import psutil
from osgeo import gdal, osr
from datetime import datetime

# Initialize EPSG 3031 projection
outputEPSG = 3031
Psrs = osr.SpatialReference()
Psrs.ImportFromEPSG(outputEPSG)

# Function to extract date from filename
def extract_date(filename):
    return datetime.strptime(filename[12:20], "%Y%m%d")

# threshold_days = 2  # Maximum allowed separation in days #####################################
threshold_min = 11.9
threshold_max = 12.1
allPairs = []  # Store valid pairs

def get_valid_data_extent(ds):
    """Compute the bounding box of the valid (non-NaN) data in a GDAL dataset."""
    band = ds.GetRasterBand(1)
    nodata = band.GetNoDataValue()
    
    arr = band.ReadAsArray()
    
    # Create a valid mask: Exclude nodata and NaN values
    if nodata is not None:
        mask = (arr != nodata) & ~np.isnan(arr)
    else:
        mask = ~np.isnan(arr)  # Assuming float data with NaNs

    if not np.any(mask):  # No valid data
        return None

    # Get min/max row/col indices where data is valid
    row_indices = np.where(np.any(mask, axis=1))[0]
    col_indices = np.where(np.any(mask, axis=0))[0]

    if row_indices.size == 0 or col_indices.size == 0:
        return None  # No valid pixels found

    min_row, max_row = row_indices[0], row_indices[-1]
    min_col, max_col = col_indices[0], col_indices[-1]

    # Convert pixel coordinates to geographic coordinates
    geotransform = ds.GetGeoTransform()
    xRes, yRes = abs(geotransform[1]), abs(geotransform[5])  # Pixel size
    min_x = geotransform[0] + geotransform[1] * min_col
    max_x = geotransform[0] + geotransform[1] * (max_col + 1)
    max_y = geotransform[3] + geotransform[5] * min_row
    min_y = geotransform[3] + geotransform[5] * (max_row + 1)

    return (min_x, min_y, max_x, max_y, xRes, yRes)


# Iterate over all unique image pairs
for i in range(len(base_filenames)):
    for j in range(i + 1, len(base_filenames)):
        base1 = base_filenames[i]
        base2 = base_filenames[j]
        
        date1 = extract_date(base1)
        date2 = extract_date(base2)
        
       # if abs((date1 - date2).days) > threshold_days:

        if not (threshold_min <= abs((date1 - date2).days) <= threshold_max):
            print('skipping this pair because separation was ' + str(abs((date1 - date2).days)))
            continue  # Skip pairs with a separation greater than threshold
        
        allPairs.append([i, j])  # Store valid pair indices
        
        geoGRD1 = os.path.join(localDataDir, base1)
        geoGRD2 = os.path.join(localDataDir, base2)

        if os.path.exists(geoGRD1) and os.path.exists(geoGRD2):
            print("Processing pair:")
            print(f"\t{base1}")
            print(f"\t{base2}")
            
            compareFold = os.path.join(localDataDir, f"ISCE3_NormProd_EW_{date1.strftime('%Y%m%d')}_{date2.strftime('%Y%m%d')}")
            os.makedirs(compareFold, exist_ok=True)
            
            DS1 = gdal.Open(geoGRD1, gdal.GA_ReadOnly)
            DS2 = gdal.Open(geoGRD2, gdal.GA_ReadOnly)
            
            georeg1 = os.path.join(compareFold, f"georeg_1_{date1.strftime('%Y%m%d')}_EPSG{outputEPSG}.tif")
            georeg2 = os.path.join(compareFold, f"georeg_2_{date2.strftime('%Y%m%d')}_EPSG{outputEPSG}.tif")
            justMade = False

            extent1 = get_valid_data_extent(DS1)
            extent2 = get_valid_data_extent(DS2)
            
            
            # Compute the INTERSECTION of the extents
            if extent1 and extent2:
                min_x = max(extent1[0], extent2[0])  # Max of min_x
                min_y = max(extent1[1], extent2[1])  # Max of min_y
                max_x = min(extent1[2], extent2[2])  # Min of max_x
                max_y = min(extent1[3], extent2[3])  # Min of max_y
                xRes = min(extent1[4], extent2[4])  # Use the finer resolution
                yRes = min(extent1[5], extent2[5])  

                
                # Check if valid intersection exists
                if min_x >= max_x or min_y >= max_y:
                    print(f"❌ No overlapping region between {base1} and {base2}. Skipping.")
                    continue
            else:
                print(f"❌ One of the images is fully NaN. Skipping.")
                continue

            if not (os.path.exists(georeg1) and os.path.exists(georeg2)):
                gdal.Warp(georeg1, DS1, format="GTiff", dstSRS=Psrs, outputBounds=(min_x, min_y, max_x, max_y), 
                          xRes=xRes, yRes=yRes, dstNodata=np.nan, 
                          outputType=gdal.GDT_Float32, creationOptions=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
                gdal.Warp(georeg2, DS2, format="GTiff", dstSRS=Psrs, outputBounds=(min_x, min_y, max_x, max_y), 
                          xRes=xRes, yRes=yRes, dstNodata=np.nan, 
                          outputType=gdal.GDT_Float32, creationOptions=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
                justMade = True

            DS1, DS2 = None, None
            
            if os.path.exists(georeg1) and os.path.exists(georeg2):
                if justMade:
                    print("Masked non-overlapping parts with NaNs.")
                    DS1 = gdal.Open(georeg1, gdal.GA_Update)
                    DS2 = gdal.Open(georeg2, gdal.GA_Update)
                    Dat1, Dat2 = DS1.ReadAsArray(), DS2.ReadAsArray()
                    
                    NBnds1, NBnds2 = DS1.RasterCount, DS2.RasterCount
                    if NBnds1 == 1:
                        Dat1 = np.expand_dims(Dat1, 0)
                    if NBnds2 == 1:
                        Dat2 = np.expand_dims(Dat2, 0)

                    Intersection = ~np.isnan(Dat1[0]) & ~np.isnan(Dat2[0])
                    if NBnds1 > 1:
                        Intersection &= ~np.isnan(Dat1[1]) & ~np.isnan(Dat2[1])

                    for di in range(NBnds1):
                        Dat1[di][~Intersection] = np.nan
                        Bnd1 = DS1.GetRasterBand(di+1)
                        Bnd1.WriteArray(np.float32(Dat1[di]))
                        Bnd1.SetNoDataValue(np.nan)
                        Bnd1.FlushCache()
                        
                        Dat2[di][~Intersection] = np.nan
                        Bnd2 = DS2.GetRasterBand(di+1)
                        Bnd2.WriteArray(np.float32(Dat2[di]))
                        Bnd2.SetNoDataValue(np.nan)
                        Bnd2.FlushCache()

                    DS1.FlushCache()
                    DS1 = None
                    DS2.FlushCache()
                    DS2 = None
                    print("✅ Trimmed to overlapping area.")
                else:
                    print("✅ Data-sets already trimmed.")
            else:
                print("❌ Failed to co-register inputs.")
        else:
            print("❌ Missing some input files.")

DS1 = None
DS2 = None
gc.collect()

print("All valid pairs within threshold:", allPairs)


Processing pair:
	S1A__EW___A_20210716T152833_HH_grd_mli_gamma0-rtc_geo_db_3031.tif
	S1A__EW___A_20210728T152834_HH_grd_mli_gamma0-rtc_geo_db_3031.tif
✅ Data-sets already trimmed.
All valid pairs within threshold: [[0, 1]]


In [4]:
# Normprod  1.3. Define Normprod functions
#This doesn't do the actual normprod algorithms - but it defines them in functions

# PARAMETERS
windows = [11, 21, 33]  # Gaussian smoothing levels

def check_raster_stats(filepath):
    """Prints min, max, mean, and std of a raster file."""
    if not os.path.exists(filepath):
        print(f"❌ ERROR: {filepath} does not exist.")
        return
    
    ds = gdal.Open(filepath, gdal.GA_ReadOnly)
    if ds is None:
        print(f"❌ ERROR: Cannot open {filepath}")
        return
    
    arr = ds.GetRasterBand(1).ReadAsArray()
    print(f"📊 Stats for {filepath}:")
    print(f"  Min: {np.nanmin(arr)}")
    print(f"  Max: {np.nanmax(arr)}")
    print(f"  Mean: {np.nanmean(arr)}")
    print(f"  Std Dev: {np.nanstd(arr)}\n")
    ds = None

def fill_nans(image):
    """Fills NaNs before filtering to prevent them from growing."""
    nan_mask = np.isnan(image)
    
    if not np.any(nan_mask):  # If no NaNs, return original
        return image

    print("🔧 Filling NaNs before filtering...")

    # Simple approach: replace NaNs with the local mean of valid pixels
    mean_value = np.nanmean(image)
    image_filled = np.where(nan_mask, mean_value, image)

    return image_filled

# DoG is waaaaay wider than sigma. Like if sigma is 11 is goes out to like 33... Bad effects on the Tony algorithm. 
def compute_dog(image_path, output_path, sigma):
    """Computes the Difference of Gaussians (DoG) while preventing NaN spread."""
    if os.path.exists(output_path):
        print(f"✅ Skipping, {output_path} already exists.")
        return output_path

    ds = gdal.Open(image_path, gdal.GA_ReadOnly)
    if ds is None:
        print(f"❌ ERROR: Cannot open {image_path}")
        return None

    driver = gdal.GetDriverByName("GTIFF")

    # Read the single-band raster
    band = ds.GetRasterBand(1).ReadAsArray()

    # Fill NaNs before applying Gaussian filter
    band_filled = fill_nans(band)

    # Compute Gaussian smoothing on the filled data
    smoothed = gaussian_filter(band_filled, sigma=sigma)

    # DoG = Original - Smoothed
    dog = band - smoothed

    # Save DoG image
    out_ds = driver.Create(output_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32,
                           options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
    out_ds.SetGeoTransform(ds.GetGeoTransform())
    out_ds.SetProjection(ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(dog)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

    ds = None
    print(f"✅ Saved DoG image: {output_path}")
    return output_path

# Hmmmm it seems DoG is quite different from Tony's original boxcar.... Let's define a boxcar filter too.
def compute_boxcar_diff(image_path, output_path, width):
    """Computes the difference from 2D boxcar smoothing (boxcar DoG-like) while preventing NaN spread."""
    if os.path.exists(output_path):
        print(f"✅ Skipping, {output_path} already exists.")
        return output_path

    ds = gdal.Open(image_path, gdal.GA_ReadOnly)
    if ds is None:
        print(f"❌ ERROR: Cannot open {image_path}")
        return None

    driver = gdal.GetDriverByName("GTIFF")

    band = ds.GetRasterBand(1).ReadAsArray()

    # Fill NaNs before filtering
    band_filled = fill_nans(band)

    # Apply 2D boxcar filter
    smoothed = uniform_filter(band_filled, size=width, mode="nearest")

    # Difference from smoothed version
    diff = band - smoothed

    # Save result
    out_ds = driver.Create(output_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32,
                           options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
    out_ds.SetGeoTransform(ds.GetGeoTransform())
    out_ds.SetProjection(ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(diff)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

    ds = None
    print(f"✅ Saved boxcar diff image: {output_path}")
    return output_path

def compute_local_std(image_path, output_path, width):
    """Computes the local standard deviation in a boxcar window of given width."""
    if os.path.exists(output_path):
        print(f"✅ Skipping, {output_path} already exists.")
        return output_path

    ds = gdal.Open(image_path, gdal.GA_ReadOnly)
    if ds is None:
        print(f"❌ ERROR: Cannot open {image_path}")
        return None

    driver = gdal.GetDriverByName("GTIFF")

    band = ds.GetRasterBand(1).ReadAsArray()

    # Fill NaNs before filtering
    band_filled = fill_nans(band)

    # Compute mean and mean of squares in local window
    local_mean = uniform_filter(band_filled, size=width, mode="nearest")
    local_mean_sq = uniform_filter(band_filled**2, size=width, mode="nearest")

    # Standard deviation: sqrt(E[x^2] - (E[x])^2)
    local_std = np.sqrt(local_mean_sq - local_mean**2)

    # Save result
    out_ds = driver.Create(output_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32,
                           options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
    out_ds.SetGeoTransform(ds.GetGeoTransform())
    out_ds.SetProjection(ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(local_std)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

    ds = None
    print(f"✅ Saved local std image: {output_path}")
    return output_path

def nan_safe_mean_filter(values):
    """Computes the mean of non-NaN values."""
    valid_values = values[~np.isnan(values)]
    return np.nan if valid_values.size == 0 else np.mean(valid_values)


def compute_normprod(dog1, dog2, std1, std2, output_path, output_path_std, output_path_stdmean, window_size):
    """Computes Normalized Product (NormProd) using precomputed smoothed images, with NaN-safe summation."""
    print("In compute_normprod...")
    
    if os.path.exists(output_path):
        print(f"✅ Skipping, {output_path} already exists.")
        return output_path

    ds_dog1 = gdal.Open(dog1, gdal.GA_ReadOnly)
    ds_dog2 = gdal.Open(dog2, gdal.GA_ReadOnly)
    ds_std1 = gdal.Open(std1, gdal.GA_ReadOnly)
    ds_std2 = gdal.Open(std2, gdal.GA_ReadOnly)

    if not all([ds_dog1, ds_dog2, ds_std1, ds_std2]):
        print(f"❌ ERROR: Missing input files.")
        return None

    driver = gdal.GetDriverByName("GTIFF")

    print("Reading these in..")
    # Read the single-band rasters
    dog1 = ds_dog1.GetRasterBand(1).ReadAsArray()
    dog2 = ds_dog2.GetRasterBand(1).ReadAsArray()
    std1 = ds_std1.GetRasterBand(1).ReadAsArray()
    std2 = ds_std2.GetRasterBand(1).ReadAsArray()    

    # Mean the two stds together (to start with. GABBY you might have better ideas?)
    stdmean = np.mean(np.stack([std1, std2], axis=0), axis=0)
    
    print("Saving StdMean...")
    # Save output
    out_ds = driver.Create(output_path_stdmean, ds_dog1.RasterXSize, ds_dog1.RasterYSize, 1, gdal.GDT_Float32,
                           options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
    out_ds.SetGeoTransform(ds_dog1.GetGeoTransform())
    out_ds.SetProjection(ds_dog1.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(stdmean)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

    print("Computing the actual normprod...")
    # Compute NormProd: (Im1 - smooth(Im1)) * (Im2 - smooth(Im2))
    normprod = dog1 * dog2


    print("Computing the sum of DoG1*DoG2 within small windows")
    # Apply a NaN-safe summation using generic_filter
    footprint = np.ones((window_size, window_size))  # Window for summation
    print("Starting generic_filter... about 10 mins for 11*11, 12 mins for 21*21.")
    #pdb.set_trace()
    summed_normprod = generic_filter(normprod, nan_safe_mean_filter, footprint=footprint, mode='constant', cval=np.nan)
    print("Finished generic_filter")
    
    print("Saving output...")
    # Save output
    out_ds = driver.Create(output_path, ds_dog1.RasterXSize, ds_dog1.RasterYSize, 1, gdal.GDT_Float32,
                           options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
    out_ds.SetGeoTransform(ds_dog1.GetGeoTransform())
    out_ds.SetProjection(ds_dog1.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(summed_normprod)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

    # Now also write out the divide-by-stdmean one....
    out_ds = driver.Create(output_path_std, ds_dog1.RasterXSize, ds_dog1.RasterYSize, 1, gdal.GDT_Float32,
                           options=["COMPRESS=DEFLATE", "BIGTIFF=YES"])
    out_ds.SetGeoTransform(ds_dog1.GetGeoTransform())
    out_ds.SetProjection(ds_dog1.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(summed_normprod/stdmean)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

    print("Freeing memory...")
    ds_dog1, ds_dog2, normprod, summed_normprod, stdmean = None, None, None, None, None
    print(f"✅ Saved NormProd (and STD) image: {output_path}")
    return output_path

def process_image_pair(imPair, localDataDir, outputEPSG):
    """Processes an image pair to compute DoG and Normalized Product (NormProd)."""
    base1, base2 = base_filenames[imPair[0]], base_filenames[imPair[1]]
    Date1, Date2 = base1[12:20], base2[12:20]
    
    # KEEPING compareFold unchanged
    compareFold = os.path.join(localDataDir, f"ISCE3_NormProd_EW_{Date1}_{Date2}")

    georeg1 = os.path.join(compareFold, f"georeg_1_{Date1}_EPSG{outputEPSG}.tif")
    georeg2 = os.path.join(compareFold, f"georeg_2_{Date2}_EPSG{outputEPSG}.tif")

    if not (os.path.exists(georeg1) and os.path.exists(georeg2)):
        print(f"❌ Skipping pair {imPair}: missing input files.")
        return

    for sigma in windows:
        print(f"🚀 Processing DoG and NormProd with Gaussian smoothing sigma={sigma}")

        # Compute DoG images
        dog1_output = os.path.join(compareFold, f"DoG_{sigma}_EPSG{outputEPSG}_{Date1}.tif")
        dog2_output = os.path.join(compareFold, f"DoG_{sigma}_EPSG{outputEPSG}_{Date2}.tif")

        # DoGs weren't good, boxcar was much better.
        #compute_dog(georeg1, dog1_output, sigma)
        #compute_dog(georeg2, dog2_output, sigma)
        compute_boxcar_diff(georeg1, dog1_output, sigma)
        compute_boxcar_diff(georeg2, dog2_output, sigma)

        # Also need to compute the sigmas in here too...
        std1_output = os.path.join(compareFold, f"STD_{sigma}_EPSG{outputEPSG}_{Date1}.tif")
        std2_output = os.path.join(compareFold, f"STD_{sigma}_EPSG{outputEPSG}_{Date2}.tif")
        compute_local_std(georeg1, std1_output, sigma)
        compute_local_std(georeg2, std2_output, sigma)

        print("Happy with the filtering, and have made the STDs. Starting normprod...")

        # Compute Normalized Product (NormProd)
        normprod_output = os.path.join(compareFold, f"NormProd_{sigma}_EPSG{outputEPSG}_{Date1}_{Date2}.tif")
        normprod_std_output = os.path.join(compareFold, f"NormProdStd_{sigma}_EPSG{outputEPSG}_{Date1}_{Date2}.tif")
        std_output = os.path.join(compareFold, f"StdMean_{sigma}_EPSG{outputEPSG}_{Date1}_{Date2}.tif")
        compute_normprod(dog1_output, dog2_output, std1_output, std2_output, normprod_output, normprod_std_output, std_output, sigma)



In [5]:
#Normprod 1.4. Qsub 

outputEPSG = 3031

#YOUR ipynb DIRECTORY
os.chdir('/g/data/jk72/gb4219/honours/normprod_standalone/')

cwd = os.getcwd()

# Directory to store PBS scripts and helper script
script_dir = os.path.join(cwd, "pbs_jobs")
os.makedirs(script_dir, exist_ok=True)
os.makedirs("logs", exist_ok=True)

# GABBY YOU WILL HAVE TO CHANGE THE SOURCE PATH BELOW
pbs_template = """#!/bin/bash
#PBS -P jk72
#PBS -q normal
#PBS -l walltime=03:00:00
#PBS -l mem=24GB
#PBS -l ncpus=2
#PBS -l wd
#PBS -l storage=gdata/fj7+scratch/jk72+gdata/jk72+gdata/yp75
#PBS -N img_{jobname}
#PBS -o logs/{jobname}.o
#PBS -e logs/{jobname}.e

source /g/data/jk72/gb4219/miniconda3_gabby/etc/profile.d/conda.sh
conda activate fastice

python3 {run_script_path} "{base1}" "{base2}" "{local_data_dir}" "{epsg}" "{windows}"
"""

# Write the helper script into script_dir
run_script_path = os.path.join(script_dir, "run_single_pair.py")
with open(run_script_path, "w") as f:
    f.write("""\
import sys
import ast
from process_image_pairs import process_image_pair, compute_normprod, nan_safe_mean_filter, compute_local_std, compute_boxcar_diff, compute_dog, fill_nans, check_raster_stats, cleanup_intermediate_files

base1 = sys.argv[1]
base2 = sys.argv[2]
data_dir = sys.argv[3]
epsg = int(sys.argv[4])
windows = ast.literal_eval(sys.argv[5])

process_image_pair(base1, base2, data_dir, epsg, windows)
""")

# Loop through image pairs and submit each one
for i, imPair in enumerate(allPairs):
    jobname = f"pair_{i}"

    pbs_script_path = os.path.join(script_dir, f"job_{jobname}.pbs")
    with open(pbs_script_path, "w") as f:
        f.write(pbs_template.format(
            jobname=jobname,
            base1=base_filenames[imPair[0]],
            base2=base_filenames[imPair[1]],
            local_data_dir=localDataDir,
            epsg=outputEPSG,
            windows=windows,
            run_script_path=run_script_path
        ))

    # Uncomment if you want to debug interactively before submission
    # pdb.set_trace()

    subprocess.run(["qsub", pbs_script_path])

154347451.gadi-pbs
